# Object Detection using YOLOv5

This notebook implements object detection using YOLOv5, a state-of-the-art detection model. We'll use the pre-trained YOLOv5s model, which offers a good balance between speed and accuracy.


In [ ]:
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO
import os

# Set up matplotlib for larger figures
plt.rcParams['figure.figsize'] = [15, 10]

# Create necessary directories
os.makedirs('weights', exist_ok=True)
os.makedirs('images', exist_ok=True)
os.makedirs('output', exist_ok=True)

## Load the YOLOv5 Model

We'll load the pre-trained YOLOv5s model. Make sure you've downloaded the weights file (`yolov5s.pt`) and placed it in the 'weights' directory.

In [ ]:
# Check if weights file exists
WEIGHTS_PATH = 'weights/yolov5s.pt'

if not os.path.exists(WEIGHTS_PATH):
    print("Error: YOLOv5 weights file not found!")
    print("Please download yolov5s.pt and place it in the 'weights' directory")
else:
    print("YOLOv5 weights found successfully!")
    
# Load the model
model = YOLO(WEIGHTS_PATH)

# Set device (CPU/GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

In [3]:
def process_image(image_path, conf_threshold=0.25):
    """
    Process an image using YOLOv5 for object detection
    
    Args:
        image_path (str): Path to the input image
        conf_threshold (float): Confidence threshold for detections (0-1)
        
    Returns:
        None (displays the results)
    """
    # Check if image exists
    if not os.path.exists(image_path):
        print(f"Error: Image not found at {image_path}")
        return
    
    # Read the image
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Perform detection
    results = model(image, conf=conf_threshold)
    
    # Plot results
    plt.figure(figsize=(15, 10))
    
    # Original image
    plt.subplot(1, 2, 1)
    plt.title('Original Image')
    plt.imshow(image)
    plt.axis('off')
    
    # Detection results
    plt.subplot(1, 2, 2)
    plt.title('Detection Results')
    
    # Get the plotted image with detections
    result_plot = results[0].plot()
    result_plot = cv2.cvtColor(result_plot, cv2.COLOR_BGR2RGB)
    plt.imshow(result_plot)
    plt.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Print detection information
    print("\nDetections:")
    for r in results:
        for box in r.boxes:
            cls = int(box.cls[0])
            conf = float(box.conf[0])
            class_name = model.names[cls]
            print(f"{class_name}: {conf:.2f}")
            
    # Save the result
    output_path = os.path.join('output', os.path.basename(image_path))
    cv2.imwrite(output_path, cv2.cvtColor(result_plot, cv2.COLOR_RGB2BGR))
    print(f"\nSaved result to: {output_path}")

In [4]:
def batch_process_directory(directory='images', conf_threshold=0.25):
    """
    Process all images in a directory
    
    Args:
        directory (str): Path to directory containing images
        conf_threshold (float): Confidence threshold for detections
    """
    # Get all image files
    image_extensions = ('.jpg', '.jpeg', '.png', '.bmp')
    image_files = [f for f in os.listdir(directory) 
                   if f.lower().endswith(image_extensions)]
    
    if not image_files:
        print(f"No images found in {directory}")
        return
    
    print(f"Found {len(image_files)} images")
    
    # Process each image
    for image_file in image_files:
        image_path = os.path.join(directory, image_file)
        print(f"\nProcessing: {image_file}")
        process_image(image_path, conf_threshold)

## Process Single Image

To process a single image, use the following code (replace 'image_path' with your image path):

In [ ]:
# Process a single image
image_path = 'images/test1.jpg'  # Replace with your image path
process_image(image_path, conf_threshold=0.25)

## Process All Images in Directory

To process all images in your images directory:

In [ ]:
# Process all images in the directory
batch_process_directory('images', conf_threshold=0.25)